# Operations Research - Case Study on Hospital Emergency Department System
*Given the following, model/simulate the ED system (Discrete Event Simulation with SimPy) to evaluate operational system "performance":*
- The existing system is struggling with its volume of patients. 
- The predicted increase in volume is going to push the existing system past its breaking point. 
- Discuss how simulation (with SimPy) can be used to **(1) evaluate the system performance** and **(2) design the appropriate system capacity**.

We will evaluate the performance of ED via the following criteria: 
1. Average total wait time for each patient with current resources/system capacity
2. Average wait time for "fast-track" patients - To measure efficiency for quick discharge
3. Average wait time for "serious" patients whom are discharged
4. Average wait time for "serious" patients whom are transferred. 

**Assumptions:**
- Doctors are in charge of diagnosis and nurses performing lab-test + preparing medications
- Fast-track team only has 1 doctor and 1 nurse
- Main ED team has 5 doctors and 5 nurses
- Patients no need to wait for doctors if they are available (after collecting their queue numbers)
- Patients will definitely need to collect medications before being discharged.

**Algorithm Brainstorming (Simplest):** 
*How does the process run in real-life? Think of steps a patient would have to go through in an ED visit.*

*Entity - Patient, Resources - Doctors and Nurses*
- Arrive at ED, get a queue no. 50% chance of either (1) Fast-track or (2) Main ED
1. Fast-Track Patient
    - [Fast Track] See doctor for diagnosis and treatment. Determine no. of lab-tests
    - [Fast Track] If lab-test, wait for lab-test -> Queue for diagnosis -> Discharge
    - [Fast Track] If healthy, Discharge. 
2. Main ED Patient
    - ...

 
**Useful links:**
- Documentation & Examples: https://simpy.readthedocs.io/en/latest/simpy_intro/index.html#
- Demo: https://realpython.com/simpy-simulating-with-python/#creating-the-environment-class-definition

In [ ]:
# Simple set-up (for fast track patients with max 1 lab-test scheduled)
import simpy
import random
import statistics

In [2]:
# hold wait times
wait_times = []

### Firstly, we create a class for fast-track patient admissions
- Set-up resource: Doctors & Nurses
- Set-up processes: Consult doctor, wait for medication, (if any) wait for lab-test result.

In [13]:
# set-up input variables
MIN_CONSULT = 1
MAX_CONSULT = 10 # min & max wait times to consult a doctor
# ...

# Create class for fast-track admissions + set-up resources and actions/processes (as class methods)
class Hospital(object): 
    def __init__(self, env, num_doctors, num_nurses): 
        self.env = env
        self.doctor = simpy.Resource(env, num_doctors)
        self.nurse = simpy.Resource(env, num_nurses)

    # consulting a doctor
    def consult(self, patient): 
        yield self.env.timeout(random.randint(1, 10))
    
    # wait for medications
    def medication(self, patient): 
        yield self.env.timeout(random.randint(5, 15))

    # wait for lab-test
    def lab(self, patient): 
        yield self.env.timeout(random.randint(5, 20))

### Next, after setting-up the environment in the fast_track class, we can keep track of the events of a Patient
- Request a resource (doctor or nurse)
- Wait for process to complete
- Leave process for the next one or get discharged!

In [6]:
def visit_hospital(env, patient, Hospital):
    # Track time patient arrive at hospital
    arrival_time = env.now

    with Hospital.doctor.request() as request:
        yield request
        yield env.process(Hospital.consult(patient))
    
    # If lab-test needed, wait for lab-test -> Go for 2nd consult
    if random.choice([True, False]):
        # Wait for lab-test
        with Hospital.nurse.request() as request:
            yield request
            yield env.process(Hospital.lab(patient))
        # Go for 2nd consult
        with Hospital.doctor.request() as request:
            yield request
            yield env.process(Hospital.consult(patient))
        
    # Wait for medications
    with Hospital.nurse.request() as request:
        yield request
        yield env.process(Hospital.medication(patient))

    # Append wait time for patient
    wait_times.append(env.now - arrival_time)

### Next up! We define a function to run the simulation!
*Start by creating an instance of the hospital, then generate patients until the simulation stops!*

In [7]:
# Define simulation function
def run_hospital(env, num_doctors, num_nurses): 
    # create Hospital instance
    hospital = Hospital(env, num_doctors, num_nurses)

    # start simulation with 3 patients waiting to be admitted - ?
    for patient in range(3):
        env.process(visit_hospital(env, patient, hospital))

    # wait before generating new patient visit
    while True:
        yield env.timeout(0.20)  # wait before generating a new person

        # add and process new patient
        patient += 1
        env.process(visit_hospital(env, patient, hospital))

### Define and run main() function.
- Set-up random seed for reproducibility of results
- Input resource parameters - No. of doctors and nurses. 
- Create the environment and save it as the variable env, which will move the simulation through each time step.
- Instantiate and Run the Process - run_hospital(), which creates hospital environment and generate patients to run within time limit.
- View average wait time for fast-track patients.

In [16]:
# Define func to get average wait times
def calculate_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    # Pretty print the results
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

In [ ]:
# Setup
random.seed(42) # ?Check
num_doctors, num_nurses = 1, 1

# Run the simulation
env = simpy.Environment()
# Instantiate and Run simulation. 
env.process(run_hospital(env, num_doctors, num_nurses))
env.run(until=120)

# View the results
mins, secs = calculate_wait_time(wait_times)
print(
    "Running simulation...",
    f"\nThe average wait time is {mins} minutes and {secs} seconds.",
)

Running simulation... 
The average wait time is 62 minutes and 38 seconds.
